In [1]:
import os
import pandas as pd

domains = ["cooking", "diy"]

k = 10

for domain in domains:

    run_files_path = f"measurements/{domain}/run_files"
    run_files = os.listdir(run_files_path)

    annotation_set = set()

    for file in run_files:
        with open(os.path.join(os.getcwd(), run_files_path, file), "r") as f:
            # runs = [run for run in f]
            for run in f:
                run_components = " ".join(run.split(' ')[0:3])
                # print(run)
                if int(run.split(' ')[3]) <= k:
                    annotation_set.add(run_components)

    empty_annotations = sorted(annotation_set)

    queries = pd.read_csv(f'queries/{domain}.csv')

    print("annotations per topic", len(empty_annotations) / 100)

    annotations_list = []

    for annotations in empty_annotations:
        query_id, _, taskgraph_id = annotations.split(' ')
        # print(query_id)
        raw_query = queries[queries["id"] == query_id].iloc[0]["raw query"]
        annotations_list.append({
            "query-id": query_id,
            "raw query": raw_query,
            "taskgraph-id": taskgraph_id,
        })

    empty_annotations_path = f"measurements/{domain}/empty_annotations"
    df = pd.DataFrame(annotations_list)
    df.to_csv(os.path.join(os.getcwd(), empty_annotations_path, f"updated-empty_annotations-{k}.csv"), index=False)
# queries["id"]

# with open(os.path.join(os.getcwd(), empty_annotations_path, 'combined_annotations.csv'), "r") as f:
        


annotations per topic 57.24
annotations per topic 33.46


In [9]:
import json
from draw import get_taksgraph_visualization
from google.protobuf.json_format import MessageToDict
import json
from google.protobuf.json_format import Parse
from pyserini.search import LuceneSearcher

from taskmap_pb2 import TaskMap

def get_taskgraph(id, domain):
    # print(os.path.join(os.getcwd(), "indexes", domain, "system_index_sparse"))
    taskmap_index_path = os.path.join(os.getcwd(), "indexes", domain, "system_index_sparse")
    searcher = LuceneSearcher(index_dir=taskmap_index_path)
    id = id.replace('\n','')
    doc = searcher.doc(docid=id)
    taskmap_json = json.loads(doc.raw())['recipe_document_json']
    taskmap = Parse(json.dumps(taskmap_json), TaskMap())
    return taskmap

/home/ubuntu/task-search-quality/env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-02-10 23:36:54.634042: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-10 23:36:55.926002: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-02-10 23:36:55.926108: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_

In [29]:
# get missing annotations
import csv 
updated_annotations_path = "measurements/cooking/empty_annotations/updated-empty_annotations-10.csv"
# new_annotations_path = "measurements/diy/empty_annotations/empty_annotations-10.csv"
qrels_path = "/home/ubuntu/task-search-quality/measurements/judgments/cooking.qrels"

judgments_csv = "measurements/cooking/empty_annotations/judgments.csv"
queries = pd.read_csv('queries/cooking.csv')

with open(updated_annotations_path, "r") as f:
    updated_annotations = [line for line in f]
    print(updated_annotations[1])
    print(len(updated_annotations))
    
with open(qrels_path, "r") as f:
    annotations_done = [line.strip().split(" ") for line in f]
    annotations_dict = {}
    for a in annotations_done:
        q_id, _, task_id, rel = a
        annotations_dict[(q_id, task_id)] = rel
    # print(annotations_done[1])
    # print(len(annotations_done))
    keys = [key for key in annotations_dict.keys()]
    print(keys[0])

annotations_list = []
for l in updated_annotations[1:]:
    l = l.strip()
    q_id, task_id = l.split(',')[0], l.split(',')[2]
    # print(q_id, task_id)
    print((q_id, task_id))
    if (q_id, task_id) in annotations_dict:
        rel = annotations_dict[(q_id, task_id)]
        del annotations_dict[(q_id, task_id)]
    else:
        rel = ''
    taskmap = get_taskgraph(task_id, "cooking")
    query_raw = queries[queries["id"] == q_id].iloc[0]
    # print(query_raw)
    annotations_list.append([q_id, task_id, query_raw["raw query"], taskmap.title, taskmap.source_url, rel,])

for (q_id, task_id), rel in annotations_dict.items():
    taskmap = get_taskgraph(task_id, "cooking")
    annotations_list.append([q_id, task_id, query_raw, taskmap.title, taskmap.source_url, rel,])

print(len(annotations_list))

with open(judgments_csv,'w') as result_file:
    wr = csv.writer(result_file, delimiter=',')
    wr.writerows(annotations_list)
# print(diff_annotations)
    
# diff_annotations =  list(updated_annotations.difference(annotations))
# diff_annotations = [a.strip().split(',') for a in diff_annotations]


# len(diff_annotations)


query-0,how to spatchcock a turkey,cooking+recipe1m+06c38e07b3533c34738b86f379aafe5c

5725
('query-0', 'cooking+recipe1m+0c118e37afefd6deb9ba92bd544a304b')
('query-0', 'cooking+recipe1m+06c38e07b3533c34738b86f379aafe5c')
('query-0', 'cooking+recipe1m+0c118e37afefd6deb9ba92bd544a304b')
('query-0', 'cooking+recipe1m+104fe308f78ba2bd4512b9bcf15b86cc')
('query-0', 'cooking+recipe1m+10ce0716fe29e6db11f84f9710e0dec8')
('query-0', 'cooking+recipe1m+197ad788ba77edd3dbdd347e519974e1')
('query-0', 'cooking+recipe1m+1cf44b9a87c3de4c1737e642d6a2a6bb')
('query-0', 'cooking+recipe1m+1d85dfb6877616b46e58860e65432b7d')
('query-0', 'cooking+recipe1m+1e205a8682a00efa1baf76536ef65ff7')
('query-0', 'cooking+recipe1m+1f8b10db4893b0e87c801a036c8f392d')
('query-0', 'cooking+recipe1m+242b73a56b90e942b61d19ff2200e02d')
('query-0', 'cooking+recipe1m+2ec94e5fd05bfe4da8a234015826217a')
('query-0', 'cooking+recipe1m+368ce1c4b71ae8147e0fd67bbc26684a')
('query-0', 'cooking+recipe1m+3e6e20ac5dd03913760c3563a226091c')

In [32]:
import sys

sys.path.insert(0, 'compiled_protobufs')

In [36]:
from models_indexes.marqo_model import MarqoModel
from models_indexes.abstract_model import AbstractModel

models = {}

# for domain in  ["DIY", "COOKING"]:
for domain in  ["DIY"]:
    models[domain] = {
        # "bm25" : BM25Model(domain = domain),
        # "bm25+rm3" : BM25Model(domain = domain, rm3=True),
        # "bm25+t5" : BM25Model(domain = domain, t5=True),
        # "bm25+rm3+t5" : BM25Model(domain = domain, rm3=True, t5=True),
        # "ance": AnceModel(domain = domain),
        # "colbert": ColbertModel(domain = domain),
        "marqo": MarqoModel(domain = domain),
    }
    
models["DIY"]["marqo"].search("fix a fridge")
# models["DIY"]["marqo"].search("fridge")
# models["DIY"]["marqo"].get_stats()
models["DIY"]["marqo"].get_single_document("diy+wikihow-offline+4e3c05e275eed0c1b2572ac770139c52")
# print("Creating run files ")
# for domain, index_models in models.items():
#     qs = queries[domain]
#     print(f"DOMAIN {domain}")
#     for model_name, model in index_models.items():
#         model.convert_search_results_to_run(qs)

{'hits': [], 'processingTimeMs': 35, 'query': 'fix a fridge', 'limit': 10}

In [17]:
run_file = "/home/ubuntu/task-search-quality/measurements/diy/run_files/ance.run"
qrels_file = "/home/ubuntu/task-search-quality/measurements/judgments/diy.qrels"

def find_missing_runs(run_file, qrels_file, k = 10):
    with open(run_file, "r") as f:
        runs = [line.strip().split(" ") for line in f]
        for run in runs:
            run[3] = int(run[3])
            run[4] = float(run[4])
        runs = [run for run in runs if run[3] <= 10]
        runs = {(run[0],run[2]) for run in runs}
    # print(runs[:10])
        
    with open(qrels_file, "r") as f:
        qrels = [line.strip().split(" ") for line in f]
        qrels = {(qrel[0], qrel[2]) for qrel in qrels}
    # print(qrels[:10])
    dif = runs.difference(qrels)
    for d in dif:
        print(f'{d[0]} Q0 {d[1]} 0')
    print(dif)
    
find_missing_runs(run_file, qrels_file)
        

query-6 Q0 diy+wikihow-offline+3f5716a7d0530fcb662a9cc257419c7f 0
query-43 Q0 diy+wikihow-offline+5275a81226a3568e6b8559c77ce16c98 0
{('query-6', 'diy+wikihow-offline+3f5716a7d0530fcb662a9cc257419c7f'), ('query-43', 'diy+wikihow-offline+5275a81226a3568e6b8559c77ce16c98')}
